Setup

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
#from sklearn.cross_validation import train_test_split 
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
%matplotlib inline

df = pd.read_csv("qudditch_training_0_1.csv",)

C:\Users\Vaneeza Kamil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (27,30,34,35,36,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
#data = (df[~np.isnan(df).any(axis=1)])

print(df.shape)
data = df.values
X0 = data[0:101267,2:47]
X1 = np.delete(X0,[3,8,18,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43],1)
#X1:house, gender, age, foul_type_id, game_move_id,penalty_id,game_duration,move_speciality,num_game_moves,
#   num_game_losses,num_practice_sessions,num_games_satout,num_games_injured,num_games_notpartof,player_type,
#   num_games_won,stooging?,body_blow?,power_play?,sloth_grip_roll?,change,snitch_caught

Y0 = data[:,-1]
print(X1.shape)

#Y0 = np.array(Y0)
Y = Y0=='YES'
Y = Y.reshape((Y.shape[0],1))
print(Y.shape)

gryffindor = np.zeros((X1.shape[0],1))
hufflepuff = np.zeros((X1.shape[0],1))
ravenclaw = np.zeros((X1.shape[0],1))
slytherin = np.zeros((X1.shape[0],1))

female = np.zeros((X1.shape[0],1))
male = np.zeros((X1.shape[0],1))

print(X1[0])

"""
if you want, we can use this to get rid of the Yes/No in the csv and turn them into 1/0:
I feel like 1/0 
"""
for row in data:
    for column in row:
        if column == "Yes" or "YES":
            column = 1
        elif column == "No" or "NO":
            column = 0

for row in range(len(X1)):
    if X1[row][0]=="Gryffindor":
        gryffindor[row]=True
    elif X1[row][0]=="Hufflepuff":
        hufflepuff[row]=True
    elif X1[row][0]=="Ravenclaw":
        ravenclaw[row]=True
    elif X1[row][0]=="Slytherin":
        slytherin[row]=True
        
    if X1[row][1]=="Female":
        female[row]=True
    elif X1[row][1]=="Male":
        male[row]=True
        
    if X1[row][-1]=="Yes":
        X1[row][-1]=1
    else:
        X1[row][-1]=0
        
    if X1[row][-2]=="Ch":
        X1[row][-2]=1
    else:
        X1[row][-2]=0
        
print(X1[0])
        
X1 = np.hstack((X1,gryffindor,hufflepuff,ravenclaw,slytherin,female,male))
X = np.delete(X1,[0,1],1)
print(X[0])

print(X[0:20,-6:-2])
print(X[0:20,-2:])

(101268, 48)
(101267, 20)
(101268, 1)
['Gryffindor' 'Female' 11.0 6.0 25.0 1.0 1.0 '38' 41.0 0.0 1.0 0.0 0.0 0.0
 'Beater' 1.0 'None' '0' '0' 0.0]
['Gryffindor' 'Female' 11.0 6.0 25.0 1.0 1.0 '38' 41.0 0.0 1.0 0.0 0.0 0.0
 'Beater' 1.0 'None' '0' 0 0]
[11.0 6.0 25.0 1.0 1.0 '38' 41.0 0.0 1.0 0.0 0.0 0.0 'Beater' 1.0 'None'
 '0' 0 0 1.0 0.0 0.0 0.0 1.0 0.0]
[[1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 0.0]]
[[1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]]


In [14]:
trainX=X[0:50634]
testX=X[50534:101267]

Logistic Regression

In [15]:
"""this is from the slides for logisitic regression:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver=‘lbfgs',multi_class='multinomial')
clf.fit(X_train, y_train)
yhat_test = clf.predict(X_test)
score = clf.score(X_test, y_test) 
"""


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report


#helpful link for log reg: https://towardsdatascience.com/logistic-regression-a-simplified-approach-using-python-c4bc81a87c31

#train test split:
#preprocessing to scale the data and use train_test_split to split the data (70% train and 30% test)
x_scale = preprocessing.scale(X)
x_train, x_test, y_train, y_test = train_test_split(x_scale, Y)
print(x_train.shape)
print(y_train.shape)

"""
#this trains and predicts
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

#check precision, recall, f1-score using classification report
print(classification_report(y_test,predictions))

"""



"""another option found in hw5"""
#log reg with lasso(l1) regularization
# Initialize an empty list to store values of training set accuracy.
acc_train_logreg = [] 
# Initialize an empty list to store values of test set accuracy.
acc_test_logreg = []
# Initialize an empty list to store different values of parameter 'c'.
c_logreg = []
# Complete the function logreg that accepts 'c' as a parameter, which is used to create logreg model 
# with different values of C.
def logreg_model(c , X_train, Y_train, X_test, Y_test):
    # Create an object of logistic regression model using linear_model.
    # Pass the value of penalty as 'L1'. By default, it is 'L2'.
    # Pass the value of C = c. Note that C is the inverse of lambda. So, small value of C i.e. b/w 0 and 1 
    # means stronger regularization and large value means less regularization.
    # Also, in sklearn, L1 is only supported with solver = 'saga'. Solver is the type of optimization algorithm like GDA or
    # SGDA, which is to be used. So, 'saga' is another algorithm like that. Pass the value of solver as 'saga'

    #Create the Logistic Regression model object as described above and save it to logreg
    logreg = linear_model.LogisticRegression(C=c,penalty='l1', warm_start=True, solver='saga')
    
    #Fit the model on the training set
    logreg.fit(X_train, Y_train)
    
    #Find the prediction on training set
    Yhat_train = logreg.predict(X_train)
    
    # Adding training accuracy to acc_train_logreg
    acc_train = np.mean(Yhat_train == Y_train)
    acc_train_logreg.append(acc_train)
    print("Accuracy on training data = %f" % acc_train)
    
    #Find the prediction on test set
    Yhat_test = logreg.predict(X_test)
    
    # Adding testing accuracy to acc_test_logreg
    acc_test = np.mean(Yhat_test == Y_test)
    acc_test_logreg.append(acc_test)
    print("Accuracy on test data = %f" % acc_test)
    
    # Appending value of c for graphing purposes
    c_logreg.append(c)
    

    
    
#pick an appropriate value for c and CHANGE the next line:
c = 10000
logreg_model(c,x_train,y_train,x_test,x_train)

ValueError: could not convert string to float: 'Steady'

Support Vector Machines

Neural Networks